In [1]:
from pathlib import Path
import sys, os
from dotenv import load_dotenv

root_path = Path.cwd() / "qa" / "mt"
print(f"{root_path=}")
sys.path.append(str(root_path))
env_path = root_path / "src" / "config" / ".env"
print(f"Loaded env:{load_dotenv(dotenv_path=env_path)}")
print(f"{os.environ.get('HF_HOME')=}")

root_path=PosixPath('/mnt/storage_2/scratch/pl0145-01/jsmok/qa/mt')
Loaded env:True
os.environ.get('HF_HOME')='/mnt/storage_2/scratch/pl0145-01/jsmok/cache/hf_home'


In [2]:

from src.config.configs import get_config

log_path: /mnt/storage_2/scratch/pl0145-01/jsmok/resources/logs


In [3]:
config = get_config("config1")

In [4]:
from src.lib.Embeddings.tools import get_embedding_model
embedding_model = get_embedding_model(config)

2024-06-14 03:15:59,925 - INFO - device: cuda
/mnt/storage_2/scratch/pl0145-01/jsmok/qa/mt/venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
2024-06-14 03:16:04,173 - INFO - Load pretrained SentenceTransformer: intfloat/multilingual-e5-large
/mnt/storage_2/scratch/pl0145-01/jsmok/qa/mt/venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [17]:
from src.lib.VectorDb.tools import get_vector_db
vector_db = get_vector_db(config)


In [4]:
from src.lib.common.tools import logger

In [5]:
logger.info("xd")